- 환경 변수 가져오기

In [ ]:
from dotenv import load_dotenv	# .env 파일 읽어주는 함수 load_dotenv 불러오기
import os		# os 환경변수에 접근
load_dotenv()		# .env 파일 내부 값을 환경변수로 등록

True

In [2]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_API_KEY

'pcsk_3JPLsi_KXPfeHMN4cGmKqcBVLWn9PbMN1tfnmkakiM7jZckEvJKcg2MeEC8g6ofMgars9a'

- Pinecone 클라이언트 초기화

In [ ]:
from pinecone import Pinecone,ServerlessSpec		# Pinecone 라이브러리에서 Pinecone 객체와 ServerlessSpec(서버 설정용) 불러오기
pine = Pinecone(api_key=PINECONE_API_KEY)
# pine = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))

# => Pinecone 서버에 로그인하고, 벡터DB 쓸 준비

- 인덱스 생성(서버리스)

In [ ]:
index_name = 'wiki'		# Pinecone 안에 폴더(index/데이터 저장소)를 생성

In [ ]:
pine.create_index(
	name=index_name,
	dimension=1536,						# : 벡터의 길이(차원 수)
	metric="cosine",					# 벡터들끼리 얼마나 비슷한지. (코사인 유사도) 의미가 비슷할 수록 각도가 작음.
	# -> 대부분의 AI 임베딩 모델은 cosine 사용. (두 문장의 의미적 유사도를 비교하기에 적절)
	spec=ServerlessSpec(cloud='aws',region='us-east-1')		# 서버가 어디에 만들어질지 설정
	# ㄴ cloud 어떤 클라우드에 저장할지, region 어느 지역 서버를 쓸지. (aws/us-east-1 : 기본설정. 가장 안정적)
)

# => 의미가 비슷한 데이터끼리 잘 찾게 해주고, AWS 서버에 저장해라

# "status": {
#        "ready": true,
#        "state": "Ready"					: 인덱스가 잘 만들어졌다는 의미

{
    "name": "wiki",
    "metric": "cosine",
    "host": "wiki-y1qvxtr.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

- 임베딩 객체 생성

In [ ]:
# 사용할 인덱스 가져오기
index = pine.Index(index_name)
index.describe_index_stats()				# Pinecone 인덱스의 상태 요약
# ㄴ stats : statistics(통계,요약)

c:\Users\C106\miniconda3\envs\llmEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [ ]:
from langchain_openai import OpenAIEmbeddings
enbedding = OpenAIEmbeddings(model='text-embedding-3-small')	
# 임베딩 모델의 차원(dimension) 과 생성한 인덱스의 dimension 이 같아야 합니다.
# 			ㄴ : 이 임베딩 모델이 만드는 벡터의 길이가 Pinecone 인덱스의 dimension=1536 설정과 같아야 함

# 임베딩(embedding) : 단어/문장 같은 글자를 숫자 벡터로 바꿔주는 기술(글의 의미를 숫자로 표현하는 방법)
# 			ㄴ AI 검색, 챗봇 기억 기능, 추천 시스템에 사용